<a href="https://colab.research.google.com/github/whyzhuce/XConparraison/blob/master/Spread_option_bi_SABR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pricing d'une Spread option en bi-SABR
----


> **Antonin Chaix**


Le but de ce TP est d'évaluer la spread option dont le payoff en $T$ est :

$$
(X_T-Y_T-K)^+
$$

où $X$ et $Y$ sont deux actifs. En supposant (sans perte de généralité) que le taux $r$ est constant et que les deux actifs ne distribuent pas de dividendes, les forwards de maturité $T$ associés aux actifs $X$ et $Y$ sont :

$$
F_t^X= X_t e^{r(T-t)} \qquad F_t^Y= Y_t e^{r(T-t)}
$$

Chacun de ces deux forwards est donc modélisé au moyen du modèle SABR :

$$
dF_t = \sigma_t F_t^\beta dW_t^1\\
\frac{d\sigma_t}{\sigma_t} = \alpha dW_t^2
$$

défini par les 4 paramètres :
* $\sigma_0$ (valeur initiale de la volatilité)
* $\alpha$ (volatilité de la volatilité)
* $\beta$ (exposant CEV)
* $\rho$ (correlation entre le forward et sa volatilité, i.e. entre $W^1$ et $W^2$)

A ces paramètres SABR (4 pour chaque actif, soit 8 au total), il faut ajouter :
* La corrélation entre $X$ et $Y$
* La corrélation entre $X$ et la volatililité de $Y$
* La corrélation entre $Y$ et la volatililité de $X$
* La corrélation entre les volatilités de $X$ et de $Y$


## Simulation des trajectoires dans le modèle bi-SABR

Pour simuler les deux forwards en SABR nous avons besoin de simuler 4 aléas corrélés (forward + vol de X, idem pour Y). Nous utilisons donc la décomposition de Cholesky comme expliqué dans [cet autre TP](https://colab.research.google.com/drive/1K77rCifOCeF9jdQkGICqLWnkTSwpAjUD?usp=sharing).

Pour plus de précisions sur le modèle SABR et la façon d'implémenter le Monte Carlo associé, se reporter à [ce TP](https://colab.research.google.com/drive/1NhQqGLtLBl1ACUDOKEJkbUzi7cMbbiax?usp=sharing).

In [ ]:
import numpy as np
import math
from scipy.linalg import cholesky

# Option maturity = MC Horizon
T = 1

# underlyings params
r = 0.02
discount = math.exp(-r*T)

X0 = 100.
fwd_X0 = X0 / discount

Y0 = 100.
fwd_Y0 = Y0 / discount

# SABR params
X_sigma0 = 0.20 # vol initiale
X_alpha = 0.60 # volvol
X_rho = -0.50 # correl fwd/vol
X_beta =  1. # exposant CEV

Y_sigma0 = 0.25 # vol initiale
Y_alpha = 0.50 # volvol
Y_rho = -0.60 # correl fwd/vol
Y_beta =  1. # exposant CEV

# other correls
rho_XY = 0.5
rho_volXvolY = 0.8
rho_XvolY = -0.4
rho_YvolX = -0.5

# Cholesky decomposition of correl matrix
correl = np.array( [[1.,        X_rho,        rho_XY,    rho_XvolY   ],
										[X_rho,     1.,           rho_YvolX, rho_volXvolY],
										[rho_XY,    rho_YvolX,    1.,        Y_rho       ],
										[rho_XvolY, rho_volXvolY, Y_rho,     1.          ]] )


L = cholesky(correl, lower=True)

# MC params
Nsimul = 2*10**5
Nsteps = 100

# some display
print("INPUTS | {:,} simulations | {} time steps | T = {} | ".format(Nsimul, Nsteps, T).replace(',', ' '))
print("\nParams SABR X :", X_sigma0, X_alpha, X_rho, X_beta)
print("Params SABR Y :", Y_sigma0, Y_alpha, Y_rho, Y_beta)
print("Correl X / Y :", rho_XY)
print("Correl vol X / vol Y :", rho_volXvolY)
print("Correl X / vol Y :", rho_XvolY)
print("Correl Y / vol X :", rho_YvolX)

# façon plus compacte de générer les lois corrélées
# normal = np.empty(shape=(4, Nsteps, Nsimul))
# for i in range(Nsteps) :
#  	normal[:,i,:] = L.dot(np.random.normal(0, 1, (4, Nsimul)))

norm1 = np.random.normal(0, 1, (Nsteps, Nsimul))
norm2 = np.random.normal(0, 1, (Nsteps, Nsimul))
norm3 = np.random.normal(0, 1, (Nsteps, Nsimul))
norm4 = np.random.normal(0, 1, (Nsteps, Nsimul))

norm_X    = norm1
norm_volX = L[1,0] * norm1 + L[1,1] * norm2
norm_Y    = L[2,0] * norm1 + L[2,1] * norm2 + L[2,2] * norm3
norm_volY = L[3,0] * norm1 + L[3,1] * norm2 + L[3,2] * norm3 + L[3,3] * norm4

# precalcs
dt = T / Nsteps
sqrt_dt = math.sqrt(dt)

# forward on time steps x simulations
fwd_X = np.empty(shape=(Nsteps+1, Nsimul))
fwd_X[0,:] = fwd_X0

# vol on time steps x simulations
sigma_X = np.empty(shape=(Nsteps+1, Nsimul))
sigma_X[0,:] = X_sigma0

# forward on time steps x simulations
fwd_Y = np.empty(shape=(Nsteps+1, Nsimul))
fwd_Y[0,:] = fwd_Y0

# vol on time steps x simulations
sigma_Y = np.empty(shape=(Nsteps+1, Nsimul))
sigma_Y[0,:] = Y_sigma0

# MC loop SABR
for i in range(Nsteps) :
	# X & X vol
	fwd_X[i+1,:] = fwd_X[i,:] + sigma_X[i,:] * np.power(fwd_X[i,:], X_beta) * sqrt_dt * norm_X[i,:]
	sigma_X[i+1,:] = sigma_X[i,:] * np.exp(-0.5 * X_alpha**2 * dt + X_alpha * sqrt_dt * norm_volX[i,:])
	# Y & Y vol
	fwd_Y[i+1,:] = fwd_Y[i,:] + sigma_Y[i,:] * np.power(fwd_Y[i,:], Y_beta) * sqrt_dt * norm_Y[i,:]
	sigma_Y[i+1,:] = sigma_Y[i,:] * np.exp(-0.5 * Y_alpha**2 * dt + Y_alpha * sqrt_dt * norm_volY[i,:])



INPUTS | 200 000 simulations | 100 time steps | T = 1 | 

Params SABR X : 0.2 0.6 -0.5 1.0
Params SABR Y : 0.25 0.5 -0.6 1.0
Correl X / Y : 0.5
Correl vol X / vol Y : 0.8
Correl X / vol Y : -0.4
Correl Y / vol X : -0.5


## Pricing de la spread option dans le modèle bi-SABR

On définit une petite fonction permettant de calculer/afficher prix et intervalle de confiance... et on calcule le payoff (très simple) de la spread option avant d'appeler cette fonction...

In [ ]:
# compute and display price & confidence interval
def compute_and_print_price(payoff, option_name, closed_form_price = -1) :
    price = payoff.mean()
    stdev = np.std(payoff)
    IClow = price - 1.96 * stdev / math.sqrt(Nsimul)
    ICup = price + 1.96 * stdev / math.sqrt(Nsimul)
    if (closed_form_price == -1) : print("{} = {:.4f}".format(option_name, price) )
    else :  print("{} = {:.4f} (closed-form price = {:.4f})".format(option_name, price, closed_form_price) )
    print("IC 95% = [{:.4f} ; {:.4f}]".format(IClow,ICup) )
    print("-----------------------------------------------------")
    return price

# spread option strike
K = 15

# spread option payoff
payoff_spread_option_sabr = discount * np.maximum(fwd_X[-1,:] - fwd_Y[-1,:] - K, 0.)

# MC Price
print("SPREAD OPTION | T = {} | K = {} | \n".format(T, K))
sabr_price = compute_and_print_price(payoff_spread_option_sabr, "Bi-SABR price")

SPREAD OPTION | T = 1 | K = 15 | 

Bi-SABR price = 3.7319
IC 95% = [3.6909 ; 3.7730]
-----------------------------------------------------


## Comparaison avec les prix analytiques dans les modèles Black-Scholes et normal

Nous allons avoir besoin des fonction suivantes (implémentées dans la cellule suivante) :

* Formule de Black pour un call
* Formule normale pour un call (Bachelier)
* Fonction permettant de convertir une vol Black en vol normale
* Prix d'une spread option dans le modèle normal
* Prix d'une spread option dans le modèle de Black
* Formule SABR (donnant la valeur approchée de la vol implicite dans le modèle SABR)

Le pricing de la spread option dans le modèle de Black (fonction `bs_spread_option` définie plus bas) nécessite peut-être quelques explications... On souhaite donc évaluer :


$$
e^{-rT}\mathbb E^Q\left((X_T - Y_T - K)^+\right)
$$

lorsque $X$ et $Y$ sont lognormaux. Supposons (pour alléger les notations) que $X$ et $Y$ désignent directement les forwards de maturité $T$ sur nos deux actifs. On veut donc calculer l'espérance ci-dessus en supposant que les processus $X$ et $Y$ sont deux martingales lognormales, soit :

$$
\frac{dX_t}{X_t}=\sigma_X dW_t^X \qquad \frac{dY_t}{Y_t}=\sigma_X dW_t^Y
$$

avec une corrélation $\rho$ entre les browniens $W^X$ et $W^Y$.

On peut écrire $X_T$ et $Y_T$ à partir de deux variables gaussiennes $ɛ_1$ et $ɛ_2$ centrées réduites indépendantes :

$$
X_T=X_0\exp\left(-\frac{1}{2}\sigma_X^2T+\sigma_X\sqrt{T}\left(\rho ɛ_1+\sqrt{1-\rho^2}ɛ_2\right)\right)\\
Y_T=Y_0\exp\left(-\frac{1}{2}\sigma_Y^2T+\sigma_Y\sqrt{T}ɛ_1\right)
$$

Si bien que notre espérance peut s'évaluer :

$$
\mathbb E^Q\left((X_T - Y_T - K)^+\right) = \int_{-\infty}^{+\infty}\mathbb E^Q\left(\left(X_0\exp\left(-\frac{1}{2}\sigma_X^2T+\sigma_X\sqrt{T}\left(\rho ɛ_1+\sqrt{1-\rho^2}ɛ_2\right)\right)-Y_0\exp\left(-\frac{1}{2}\sigma_Y^2T+\sigma_Y\sqrt{T}ɛ_1\right)-K\right)^+  \,\middle|\; ɛ_1 = y\right)n(y)dy
$$

où $n$ est la densité de la loi normale centrée réduite :

$$
n(x) = \frac{1}{\sqrt{2\pi}} e^{-\frac{x^2}{2}}
$$

On peut récrire notre espérance :

$$
\mathbb E^Q\left((X_T - Y_T - K)^+\right) = \int_{-\infty}^{+\infty}\mathbb E^Q\left(\left(\widehat {X_0} (ɛ_1)\exp\left(-\frac{1}{2}\widehat\sigma_X^2T+\widehat\sigma_X\sqrt{T}ɛ_2\right)-\widehat K (ɛ_1) \right)^+  \,\middle|\; ɛ_1 = y\right)n(y)dy
$$

avec :


$$
\widehat \sigma_X = \sigma_X\sqrt{1-\rho^2}
$$

$$
\widehat {X_0} (ɛ_1) = X_0 \exp\left(- \frac{1}{2}\sigma_X^2\rho^2 T+\rho\sigma_X\sqrt{T}\varepsilon_1\right)
$$

$$
\widehat K (ɛ_1) =Y_0\exp\left(-\frac{1}{2}\sigma_Y^2T+\sigma_Y\sqrt{T}ɛ_1\right)+K
$$

de sorte que que l'espérance conditionnelle sous l'intégrale se calcule au moyen de la formule de Black :

$$
\mathbb E^Q\left((X_T - Y_T - K)^+\right) = \int_{-\infty}^{+\infty}\text{BScall}\left(T, \widehat K (y), \widehat {X_0} (y), \widehat \sigma_X\right)n(y)dy
$$

Il ne reste plus qu'à calculer numériquement cette intégrale par la méthode des trapèzes et le tour est joué. Tout cela est implémenté dans la fonction `bs_spread_option` ci-dessous.


In [ ]:
from scipy.stats import norm
from scipy import optimize

# BS formula for a call / numpy compatible
def bs_call(T, K, F0, sigma) :
    sigma_sqrt_T = sigma * np.sqrt(T)
    d1 = (np.log(F0/K) + 0.5 * sigma**2 * T) / sigma_sqrt_T
    d2 = d1 - sigma_sqrt_T
    return F0 * norm.cdf(d1) - K * norm.cdf(d2)

# Normal formula for a call
def norm_call(T, K, F, sigma) :
	if (T==0 or sigma==0) : return max(F-K, 0)
	sigma_sqrt_T = sigma * math.sqrt(T)
	d = (F - K) / sigma_sqrt_T
	return sigma_sqrt_T * (d * norm.cdf(d) + norm.pdf(d))

# Converts a BS vol into normal vol
def bs_to_norm_vol(T, K, F, bs_vol) :
	guess = F * bs_vol
	bs_price = bs_call(T, K, F, bs_vol)
	def f(x):
		return norm_call(T, K, F, x) - bs_price
	return optimize.newton(f, guess)

# Spread option (normal model)
def norm_spread_option (T, K, F1, F2, sigma1, sigma2, rho):
	norm_vol = math.sqrt(sigma1**2 + sigma2**2 - 2*rho*sigma1*sigma2)
	return norm_call(T, K, F1-F2, norm_vol)

# Spread option (BS model)
def bs_spread_option (T, K, F1, F2, sigma1, sigma2, rho):
	nx, nstdev = 201, 6.
	x = np.linspace(-nstdev, nstdev, nx, endpoint=True)
	dx = (x[-1] - x[0]) / (nx - 1)
	sqrt_T = math.sqrt(T)
	strike = F2 * np.exp(-0.5*sigma2**2*T + sigma2*sqrt_T*x) + K
	strike = np.maximum(strike, 0.00001) # eviter strike negatif dans les cas extrêmes
	fwd = F1 * np.exp(-0.5*(sigma1*rho)**2*T + rho*sigma1*sqrt_T*x)
	vol = sigma1 * math.sqrt(1.-rho**2)
	integ = bs_call(T, strike, fwd, vol) * (1./math.sqrt(2.*math.pi)) * np.exp(-0.5*x**2)
	return np.sum(dx * 0.5 * (integ[:-1] + integ[1:nx]))


# SABR lognormal vol formula from [https://github.com/ynouri/pysabr] made numpy compatible
def sabr_vol (T, K, F0, sigma0, alpha, rho, beta) :
    """
    Hagan's 2002 SABR lognormal vol expansion.
    The strike K can be a scalar or an array, the function will return an array
    of lognormal vols.
    """
    eps = 1e-07
    logfk = np.log(F0 / K)
    fkbeta = (F0*K)**(1 - beta)
    a = (1 - beta)**2 * sigma0**2 / (24 * fkbeta)
    b = 0.25 * rho * beta * alpha * sigma0 / fkbeta**0.5
    c = (2 - 3*rho**2) * alpha**2 / 24
    d = fkbeta**0.5
    v = (1 - beta)**2 * logfk**2 / 24
    w = (1 - beta)**4 * logfk**4 / 1920
    z = alpha * fkbeta**0.5 * logfk / sigma0
    tmp = sigma0 * (1 + (a + b + c) * T)
    num = np.where(abs(z) > eps, z * tmp, tmp)
    den = np.where(abs(z) > eps, (d * (1 + v + w) * _x(rho, z)), d * (1 + v + w))
    return num / den

def _x(rho, z):
    """Return function x used in Hagan's 2002 SABR lognormal vol expansion."""
    a = (1 - 2*rho*z + z**2)**.5 + z - rho
    b = 1 - rho
    return np.log(a / b)


Regardons tout d'abord ce que donnent les prix BS et normal en utilisant les volatilités ATM dans ces deux modèles.

In [ ]:
# get norm & BS ATM vols for X & Y
bs_vol_X = sabr_vol(T, fwd_X0, fwd_X0, X_sigma0, X_alpha, X_rho, X_beta)
norm_vol_X = bs_to_norm_vol(T, fwd_X0, fwd_X0, bs_vol_X)

bs_vol_Y = sabr_vol(T, fwd_Y0, fwd_Y0, Y_sigma0, Y_alpha, Y_rho, Y_beta)
norm_vol_Y = bs_to_norm_vol(T, fwd_X0, fwd_Y0, bs_vol_Y)

# compute price
bs_price = discount * bs_spread_option(T, K, fwd_X0, fwd_Y0, bs_vol_X, bs_vol_Y, rho_XY)
normal_price = discount * norm_spread_option (T, K, fwd_X0, fwd_Y0, norm_vol_X, norm_vol_Y, rho_XY)

print("SPREAD OPTION | T = {} | K = {} | \n".format(T, K))
print("Spread option SABR (rappel) : {:.4f}".format(sabr_price))
print("Spread option BS (vol ATM) : {:.4f}".format(bs_price))
print("Spread option normal (vol ATM) {:.4f}".format(normal_price))
#print("Rel. error normal vs. SABR : {:.2f}%".format(100 * (normal_price - sabr_price) / sabr_price))


SPREAD OPTION | T = 1 | K = 15 | 

Spread option SABR (rappel) : 3.7319
Spread option BS (vol ATM) : 3.3978
Spread option normal (vol ATM) 3.5522


Prendre les volatitilités de $X$ et $Y$ à un strike "équivalent" améliore grandement les choses. Le principe est le suivant :

$$
(X_T-Y_T - K)^+ = (X_T-(Y_T + K))^+
$$

si bien qu'il est intéressant de calculer la volatilité de $X$ (sur son smile SABR) à un niveau de strike égal à l'espérance de $Y_T + K$ c'est à dire au strike :

$$
K_X = F_0^Y+K
$$

De la même manière on calculera la volatilité de $Y$ au strike :

$$
K_Y = F_0^X-K
$$

In [ ]:
# get norm & BS vols @ equivalent strike for X & Y
equiv_strike_X = fwd_Y0 + K
bs_vol_X = sabr_vol(T, equiv_strike_X, fwd_X0, X_sigma0, X_alpha, X_rho, X_beta)
norm_vol_X = bs_to_norm_vol(T, equiv_strike_X, fwd_X0, bs_vol_X)

equiv_strike_Y = fwd_X0 - K
bs_vol_Y = sabr_vol(T, equiv_strike_Y, fwd_Y0, Y_sigma0, Y_alpha, Y_rho, Y_beta)
norm_vol_Y = bs_to_norm_vol(T, equiv_strike_Y, fwd_Y0, bs_vol_Y)

# compute prices
bs_price = discount * bs_spread_option(T, K, fwd_X0, fwd_Y0, bs_vol_X, bs_vol_Y, rho_XY)
normal_price = discount * norm_spread_option (T, K, fwd_X0, fwd_Y0, norm_vol_X, norm_vol_Y, rho_XY)

print("SPREAD OPTION | T = {} | K = {} | \n".format(T, K))
print("Spread option SABR (rappel) : {:.4f}".format(sabr_price))
print("Spread option BS (vol @ equiv strike) : {:.4f}".format(bs_price))
print("Spread option normal (vol @ equiv strike) {:.4f}".format(normal_price))

SPREAD OPTION | T = 1 | K = 15 | 

Spread option SABR (rappel) : 3.7319
Spread option BS (vol @ equiv strike) : 3.6343
Spread option normal (vol @ equiv strike) 3.6155
